In [ ]:
# prompt: In this experiment, you will work with process calculi and formal verification techniques. For
# which you are required to:
# 1. Implement a system where two processes communicate using the π-Calculus framework,
# dynamically creating channels and exchanging messages. Ensure that the processes interact
# correctly and handle concurrent execution.
# python

import random

class Channel:
    def __init__(self, name):
        self.name = name
        self.buffer = []

    def send(self, message):
        self.buffer.append(message)

    def receive(self):
        if self.buffer:
            return self.buffer.pop(0)
        return None


class Process:
    def __init__(self, name):
        self.name = name
        self.channels = {}

    def create_channel(self):
        channel_name = f"channel_{random.randint(0, 1000)}"
        self.channels[channel_name] = Channel(channel_name)
        return channel_name

    def send(self, channel_name, message):
      if channel_name in self.channels:
          self.channels[channel_name].send(message)
      else:
          print(f"Process {self.name}: Channel {channel_name} not found.")


    def receive(self, channel_name):
      if channel_name in self.channels:
        return self.channels[channel_name].receive()
      else:
          print(f"Process {self.name}: Channel {channel_name} not found.")
          return None

    def run(self, other_process):
        # Example interaction:
        new_channel_name = self.create_channel()
        print(f"Process {self.name}: Created channel {new_channel_name}")

        other_process.channels[new_channel_name] = self.channels[new_channel_name]  # Share the channel


        self.send(new_channel_name, f"Hello from {self.name}")

        message = other_process.receive(new_channel_name)
        if message:
          print(f"Process {self.name} received: {message}")

def main():
  process1 = Process("P1")
  process2 = Process("P2")

  process1.run(process2)
  process2.run(process1)

if __name__ == "__main__":
  main()



In [ ]:
# prompt: 2. Develop a Python program that models a system of three CCS processes executing actions in
# parallel, ensuring synchronization where required. Introduce relabeling and restriction to study
# their impact on process behavior.

import random
import time

class Channel:
    def __init__(self, name):
        self.name = name
        self.buffer = []
        self.lock = False # Added a lock for synchronization

    def send(self, message):
        while self.lock: # Wait if channel is locked
            time.sleep(0.1) # Simulate waiting
        self.lock = True  # Acquire the lock
        self.buffer.append(message)
        self.lock = False  # Release the lock

    def receive(self):
        while self.lock: # Wait if channel is locked
            time.sleep(0.1) # Simulate waiting
        self.lock = True # Acquire the lock
        if self.buffer:
            message = self.buffer.pop(0)
            self.lock = False  # Release the lock
            return message
        self.lock = False # Release the lock
        return None


class Process:
    def __init__(self, name):
        self.name = name
        self.channels = {}
        self.state = "active" # Initial state

    def create_channel(self):
        channel_name = f"channel_{random.randint(0, 1000)}"
        self.channels[channel_name] = Channel(channel_name)
        return channel_name

    def send(self, channel_name, message):
        if channel_name in self.channels:
            self.channels[channel_name].send(message)
        else:
            print(f"Process {self.name}: Channel {channel_name} not found.")

    def receive(self, channel_name):
        if channel_name in self.channels:
            return self.channels[channel_name].receive()
        else:
            print(f"Process {self.name}: Channel {channel_name} not found.")
            return None

    def run(self, p2, p3):
        # Example interaction with synchronization and relabeling/restriction
        channel_p1_p2 = self.create_channel()
        channel_p1_p3 = self.create_channel()
        p2.channels[channel_p1_p2] = self.channels[channel_p1_p2]
        p3.channels[channel_p1_p3] = self.channels[channel_p1_p3]

        if self.state == "active": # Example restriction
            self.send(channel_p1_p2, f"{self.name} -> P2: Hello") # Relabeling: change the message
            self.send(channel_p1_p3, f"{self.name} -> P3: Greetings") # Relabeling: change the message

            # Example of waiting for a response
            message = self.receive(channel_p1_p2)
            if message:
                print(f"Process {self.name} received: {message}")


def main():
    p1 = Process("P1")
    p2 = Process("P2")
    p3 = Process("P3")

    # Running processes concurrently (simulated using time.sleep)
    p1.run(p2, p3)
    time.sleep(0.5)  # Simulate parallel execution
    p2.run(p1, p3)
    time.sleep(0.5)  # Simulate parallel execution
    p3.run(p1,p2)


if __name__ == "__main__":
    main()


In [ ]:
# prompt: 3. Simulate a process algebra-based load balancer where multiple clients send requests to a central
# dispatcher that distributes tasks among available workers. Verify that requests are handled
# fairly without starvation.
# using simpy

import simpy

# Constants
NUM_CLIENTS = 10
NUM_WORKERS = 3
REQUEST_INTERVAL = 1  # Time between client requests (seconds)
PROCESSING_TIME = 2  # Time to process a request (seconds)


class Client:
    def __init__(self, env, dispatcher, client_id):
        self.env = env
        self.dispatcher = dispatcher
        self.client_id = client_id
        self.action = env.process(self.run())

    def run(self):
        while True:
            # Generate a request
            request_time = self.env.now
            yield self.env.timeout(REQUEST_INTERVAL)

            # Send request to dispatcher
            with self.dispatcher.request() as req:
                yield req
                print(f"Client {self.client_id}: Request sent at {self.env.now}")
                worker_id = yield self.dispatcher.get_worker()
                # print(worker_id)
                processing_time = random.expovariate(1.0/PROCESSING_TIME)  # Vary processing times

                yield self.env.timeout(processing_time)
                print(f"Client {self.client_id}: Request processed by Worker {worker_id} at {self.env.now}")
                self.dispatcher.release_worker(worker_id)


class Dispatcher:
    def __init__(self, env, num_workers):
        self.env = env
        self.num_workers = num_workers
        self.available_workers = simpy.Resource(env, capacity=num_workers)
        self.worker_ids = list(range(num_workers))
        self.worker_queue = self.worker_ids.copy()

    def get_worker(self):
        if self.worker_queue:
            return self.worker_queue.pop(0) # FIFO
        return None # No workers available

    def release_worker(self, worker_id):
        self.worker_queue.append(worker_id)



# Setup and start the simulation
env = simpy.Environment()
dispatcher = Dispatcher(env, NUM_WORKERS)

for i in range(NUM_CLIENTS):
    Client(env, dispatcher, i)

env.run(until=50)


In [ ]:
# prompt: 4. Implement a Python-based verification system that checks whether two given finite-state
# processes are equivalent using strong bisimulation. The program should take two process
# descriptions as input and determine whether they exhibit the same external behavior.
# use simpy

import copy

def is_bisimilar(process1, process2):
    """
    Checks if two processes are bisimilar using strong bisimulation.

    Args:
        process1: A dictionary representing the first process.
        process2: A dictionary representing the second process.

    Returns:
        True if the processes are bisimilar, False otherwise.
    """

    def get_transitions(process, state):
        """
        Returns a list of transitions from a given state in a process.
        """
        return process.get(state, [])

    def explore(state1, state2, visited):
        """
        Recursively explores the state space to check for bisimulation.
        """
        if (state1, state2) in visited:
            return True

        visited.add((state1, state2))
        transitions1 = get_transitions(process1, state1)
        transitions2 = get_transitions(process2, state2)

        if len(transitions1) != len(transitions2):
          return False

        for trans1 in transitions1:
          found_match = False
          for trans2 in transitions2:
            if trans1[0] == trans2[0]: # Check for matching actions/labels
              if explore(trans1[1], trans2[1], visited):
                found_match = True
                break
          if not found_match:
            return False
        return True

    # Start exploration from initial states
    initial_state1 = list(process1.keys())[0]
    initial_state2 = list(process2.keys())[0]
    visited = set()
    return explore(initial_state1, initial_state2, visited)

# Example usage:
# Define two processes as dictionaries: state -> [(action, next_state)]
process_a = {
    "S0": [("a", "S1"), ("b", "S2")],
    "S1": [("a", "S1")],
    "S2": [("b", "S2")]
}

process_b = {
    "T0": [("a", "T1"), ("b", "T2")],
    "T1": [("a", "T1")],
    "T2": [("b", "T2")]
}

process_c = {
    "T0": [("a", "T1"), ("b", "T2")],
    "T1": [("a", "T0")], # Modified transition
    "T2": [("b", "T2")]
}


print(f"Process A and Process B are bisimilar: {is_bisimilar(process_a, process_b)}")
print(f"Process A and Process C are bisimilar: {is_bisimilar(process_a, process_c)}")


In [ ]:
# prompt: 5. Design a producer-consumer system using CCS principles, ensuring correct message passing
# and proper synchronization between the producer and the consumer while preventing
# deadlocks.
# give normal code and 2nd using simpy

import threading
import queue
import time

class Channel:
    def __init__(self, capacity):
        self.queue = queue.Queue(capacity)
        self.lock = threading.Lock()
        self.not_full = threading.Condition(self.lock)
        self.not_empty = threading.Condition(self.lock)

    def send(self, message):
        with self.not_full:
            while self.queue.full():
                self.not_full.wait()
            self.queue.put(message)
            self.not_empty.notify()

    def receive(self):
        with self.not_empty:
            while self.queue.empty():
                self.not_empty.wait()
            message = self.queue.get()
            self.not_full.notify()
            return message

class Producer(threading.Thread):
    def __init__(self, channel):
        super().__init__()
        self.channel = channel

    def run(self):
        for i in range(5):
            time.sleep(1) # Simulate some work
            self.channel.send(i)
            print(f"Producer sent: {i}")

class Consumer(threading.Thread):
    def __init__(self, channel):
        super().__init__()
        self.channel = channel

    def run(self):
        for _ in range(5):
            message = self.channel.receive()
            time.sleep(0.5) # Simulate some work
            print(f"Consumer received: {message}")


if __name__ == "__main__":
    channel = Channel(2)  # Create a channel with a capacity of 2
    producer = Producer(channel)
    consumer = Consumer(channel)

    producer.start()
    consumer.start()

    producer.join()
    consumer.join()
    print("Producer-consumer finished")


In [ ]:
# prompt: give above with simpy

from google.colab import drive
drive.mount('/content/drive')
